# Data Cleaning Using SQL for Modeling 

Importing Libraries and Loading the CSV File

In [29]:


import pandas as pd
from datetime import datetime

# Load the CSV file
file_path = 'Resources/2013_to_2023_property-sales-data.csv'

dtype_dict = {
    'FinishedSqft': 'str',
    'Lotsize': 'str',
    'Sale_price': 'str'
}
data = pd.read_csv(file_path)




C:\Users\ebehnia\AppData\Local\Temp\ipykernel_17960\1590189496.py:12: DtypeWarning: Columns (12,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Parsing and Standardizing the Date Format in Python

In [30]:
# Function to parse and standardize the date format
def parse_date(date_str):
    for fmt in ("%Y-%m-%d", "%m/%d/%Y", "%d-%b-%y", "%b-%y"):
        try:
            return datetime.strptime(date_str, fmt).strftime("%Y-%m-%d")
        except ValueError:
            pass
    return None

# Apply the function to the 'Sale_date' column
data['Sale_date'] = data['Sale_date'].apply(parse_date)

# Check for any dates that couldn't be parsed
unparsed_dates = data[data['Sale_date'].isna()]
unparsed_dates_count = len(unparsed_dates)


Clean Numeric Columns

In [31]:
# Function to clean numeric columns
def clean_numeric(column):
    # Ensure the column is a string, replace unwanted characters, and convert to numeric
    return pd.to_numeric(column.astype(str).str.replace('[\\$,]', '', regex=True).str.replace(',', ''), errors='coerce')

# Cleaning and converting the 'FinishedSqft', 'Lotsize', and 'Sale_price' columns
data['FinishedSqft'] = clean_numeric(data['FinishedSqft'])
data['Lotsize'] = clean_numeric(data['Lotsize'])
data['Sale_price'] = clean_numeric(data['Sale_price'])



Filter Data


In [32]:
data = data[data.PropType == 'Residential']
data = data[data.Sale_price >= 50000]



Drop Unnecessary Columns

In [33]:
# Dropping columns with unique values
data = data.drop(columns=['PropertyID', 'taxkey', 'Address', 'CondoProject', 'PropType'])




Fill Missing Values

In [34]:
data['Stories'] = data['Stories'].fillna(1.0)




Convert and Add Date Columns

In [35]:
# Converting 'Sale_date' to datetime with mixed format and adding sale-year and sale-month columns
data['Sale_date'] = pd.to_datetime(data['Sale_date'], errors='coerce')
data['sale_year'] = data['Sale_date'].dt.year
data['sale_month'] = data['Sale_date'].dt.month



Standardize Data

In [36]:
# Standardize column names to lowercase
data.columns = [col.lower() for col in data.columns]

# Standardizing categorical columns to consistent capitalization
data['style'] = data['style'].str.lower()
data['extwall'] = data['extwall'].str.lower()



Handle Missing Values

In [37]:
# For simplicity, we'll drop rows with missing target values (sale_price) and rows with excessive missing values.
data_cleaned = data.dropna(subset=['sale_price', 'sale_date', 'finishedsqft', 'district', 'nbhd', 'lotsize', 'hbath', 'fbath', 'rooms', 'extwall'])

# Removing Duplicate Records
data_cleaned = data_cleaned.drop_duplicates()



Check for Null Values and Reset Index

In [38]:
# Checking for null values
null_rows = data_cleaned[data_cleaned.isnull().any(axis=1)]

# Resetting the index
data_cleaned = data_cleaned.reset_index(drop=True)



In [39]:
# Saving the cleaned data back to a CSV file
cleaned_file_path = 'Resources/clean_property_data-sql2.csv'
data_cleaned.to_csv(cleaned_file_path, index=False)
